In [ ]:
!pip install transformers==4.15.0

In [ ]:
import transformers
from transformers import BertTokenizer, BartForConditionalGeneration
import collections
from collections import Counter
import os
import numpy as np

In [ ]:
# 语料已清洗，对原始语料进行读取
def readtxt(txtspath):
  txtsList=[]
  txtfiles=os.listdir(txtspath)
  for file in txtfiles:
    position = txtspath + '/' + file
    # print(position)
    with open(position,"r",encoding='UTF-8') as f:   # 打开文件
      lines = f.readlines()
      for line in lines:
        if line!='\n':
          # line = line.strip()+'[SEP]'
          txtsList.append(line)
  text = ""
  for line in txtsList:
    text += line
  return text # 返回语料段落列表

In [ ]:
# 加载字典
def load_vocab(vocab_file):
  """Loads a vocabulary file into a dictionary."""
  vocab_dicts = {}
  with open(vocab_file, "r", encoding="utf-8") as reader:
    tokens = reader.readlines()
  for index, token in enumerate(tokens):
    token = token.rstrip("\n")
    vocab_dicts[token] = index
  print('len(vocab_dicts):',len(vocab_dicts))
  return vocab_dicts

In [ ]:
# 获取bart全部21128字向量
def get_wholeEmbed(model):
  embed_weight=model.state_dict()["model.encoder.embed_tokens.weight"]
  print(embed_weight.shape)
  np.save('wholebart_charsweight.npy', embed_weight)
  return embed_weight

In [ ]:
def load_para(intToTok_path,tokToInt_path):
  # 文本的jieba词语字典
  dict_intToTok = np.load(intToTok_path,allow_pickle=True).item()
  dict_TokToInt = np.load(tokToInt_path,allow_pickle=True).item()
  return dict_intToTok,dict_TokToInt

In [ ]:
# 将分词的词语用berttokenizer分词，然后从字向量中遍历取得各向量并求平均
def get_bart_wordsweight(tokenizer,dict_intToTok,vocab_dicts,wordEmbedding):
  wordsweight=[]
  for num,tok in dict_intToTok.items():  # 结巴分词后的词语
    tok_charList = tokenizer.tokenize(tok)   # 分成字符
    tokweight=[]
    for char in tok_charList:
      try:
        index = vocab_dicts[char]
        tokweight.append(wordEmbedding[index]) 
        #print(charsweight[index])
      except:
        print(char+"不在字典中")
        if char == '\n':
          tokweight.append(wordEmbedding[vocab_dicts['[SEP]']]) 
    total = [0]*768
    if tokweight==[]:
      tokweight.append(np.array([0]*768))
    for i in range(len(tokweight)):
      total += np.array(tokweight[i])
    wordsweight.append( total / len(tokweight) )
  return np.array(wordsweight)

In [ ]:
dict_intToTok,dict_TokToInt = load_para(intToTok_path,tokToInt_path)

In [ ]:
tokenizer = BertTokenizer(vocab_file=vocab_path)
model_bart = BartForConditionalGeneration.from_pretrained("fnlp/bart-base-chinese")

In [ ]:
wordEmbedding = get_wholeEmbed(model_bart) # 获取bart全部21128字向量
vocab_dicts = load_vocab(vocab_path)

torch.Size([21128, 768])
len(vocab_dicts): 21128


In [ ]:
wordEmbedding = get_bart_wordsweight(tokenizer,dict_intToTok,vocab_dicts,wordEmbedding)